# Initial Watch Records

## 1. Web Access

### 1. Modules & Variables

In [ ]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크 (https://github.com/ultrafunkamsterdam/undetected-chromedriver)


In [1]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [2]:
# Account Variables

info = open('secure/id_pw.txt').readlines()

account_1 = info[0].split(' ')[1].rstrip('\n')
password_1 = info[1].split(' ')[1].rstrip('\n')

account_2 = info[2].split(' ')[1].rstrip('\n')
password_2 = info[3].split(' ')[1].rstrip('\n')


### 2. Web Access

In [5]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [6]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
try:
    btn_sign_in = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
    btn_sign_in.click()
except:
    pass

# 계정 이메일 입력
try:
    input_account = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
    input_account.send_keys(account_2)
except:
    pass
# [다음] 버튼 클릭해 패스워드 입력
try:
    btn_next_password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
    btn_next_password.click()
except:
    pass

# 계정 비밀번호 입력
try:
    input_password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
    input_password.send_keys(password_2)
    del password_2 # 계정 비밀번호 변수 삭제
except:
    pass

# [다음] 버튼 입력해 로그인 클릭
try:
    btn_next_sign_in = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
    btn_next_sign_in.click()
except:
    pass

In [7]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지

try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


In [31]:
driver.refresh()


### 3. Data Collection

#### 1) Variables

In [8]:
# Variables (1) - 시청기록 태그 속성


# 기본 영상 정보
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>


# 시청비율 판별 정보
nm_class_thumbnail = "OUPWA"  # 기록 우측에 있는 영상 정보 (썸네일) / tag = <div>
nm_class_percentage = "HmLFgd"  # 썸네일 하단 빨간 Percentage Bar / tag = <div>


# 시청기록 - video id
# replace(pattern, '')로 id 값 추출
id_str_pattern = "https://www.youtube.com/watch?v="

'''
WatchDate = container[data-date] / c-wiz 태그의 [data-date] 값
시청비율 확인을 위해 nm_class_RunningTime 활용
'''

In [9]:
# Variables (2) Youtube Data API


# API 실행에 필요한 모듈
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# API 정보, API Key => API build
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
DEVELOPER_KEY = open('secure/api_key.txt').readline()

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)


#### 2) Functions

In [12]:
# Function (1) - date & day

def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d')

get_date('20230203')


datetime.datetime(2023, 2, 3, 0, 0)

In [15]:
# Function (2) - Category Id & Title

""" Google API - Youtube DATA API 활용
    세부 query
    - videos().list(): id 값(watch?v="여기 표시되는 값")으로 동영상 정보 반환 -> 내부의 categoryId 호출
    - videoCategories().list(): categoryId 값(1~44)으로 카테고리 정보 반환 -> 내부의 categoryTitle 호출
"""

""" 개선 필요
        categoryTitle이 영어로 호출됨
        query 옵션에서 한국어를 호출하거나, 번역해야 한다.
"""


def get_catId(input_id):
    video = youtube.videos().list(
        part='snippet',
        id=input_id
    ).execute()

    return video['items'][0]['snippet']['categoryId']


def get_catTitle(num):
    catId = youtube.videoCategories().list(
        id=num,
        part='snippet'
    ).execute()

    try:
        result = catId['items'][0]['snippet']['title']
    except:
        result = ''

    return result


In [14]:
# Function (3) - Watched_YN classifier

""" 구현 가능 Option (1)
    - 구현법
        1. 현재 레코드의 watch_time과 이전 레코드의 watch_time의 차이값 구한다
        2. 영상 길이와 비슷하면 watched, 아닐 시 not_watched

    - 문제점
        1. 마지막으로 시청한 영상은 차이를 구할 수 없다.
        2. 길이 5분 이상 광고를 방치한 경우 시간 계산에 오류가 생긴다
"""

""" 구현 가능 Option (2) -> 채택
    - 구현법
        1. 시청 기록 레코드의 썸네일의 진행도 막대를 활용한다.
        2. 시청 완료하지 않은 영상에 표시되며, 태그 내부의 시청 비율 값을 추출한다.

    - 문제점
        1. 라이브 스트리밍의 경우 진행도 막대가 출력되지 않는다.
        2. 대충 넘기기만 해도 진행도 막대가 표시된다. 
"""


def classify_watched(content):
    running_time = content.find('div', class_=nm_class_RunningTime).text
    thumbnail = content.find('div', class_=nm_class_thumbnail)
    percentage = thumbnail.find('div', class_=nm_class_percentage)

    try:
        result = int(percentage['style'].split(":")[1][:-1]) / 100

    except:
        if len(running_time) > 5:  # 영상 길이가 1시간 이상(0:00:00)인 경우
            result = 0  # 미시청으로 간주
        else:
            result = 1  # 시청으로 간주

    return result


#### 3) Empty DataFrame

In [16]:
import pandas as pd

columns_record = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category_id', 'category_title', 'watched (%)', 'account']
df_record = pd.DataFrame(columns=columns_record)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account


In [17]:
columns_search = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_search)
df_search.head()


,searchword,search_date,search_time


#### 4) Web Data Collecting

##### 입력 날짜까지의 시청 기록 호출

In [24]:
# 날짜 입력 함수

def get_date_data():
    while True:
        try:
            year, month, day = [int(input(f"{field}를(을) 입력해주세요" )) for field in ("연도(4자리)", "월(1~2자리)", "일(1~2자리)")]
            if not (1000 <= year <= 9999 and 1 <= month <= 12 and 1 <= day <= 31):
                input("Invalid input. Wanna Try again?")
            else:
                break
        except: 
            print("Invalid input. Wanna Try again?")
        
    return f"{year:04d}{month:02d}{day:02d}"

get_date_data()

'20230102'

In [ ]:
# 입력한 개수만큼 레코드 호출

num_target_data = 10000  # int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
num_press_end = int(num_target_data / 100)

for n in range(num_press_end - 1):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END) # 페이지 내 마지막 레코드까지 출력되도록 End 키 입력
    time.sleep(2) # 전체 페이지가 뜨도록 Wait

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')

'''전체 기록 수가 입력 갯수 미만이면 반복문이 종료되지 않는다.'''

In [24]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime):  # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            channel = content.find('div', class_=nm_class_channel).text
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            watched = classify_watched(content)

            id = content.find('a')['href'].replace(id_str_pattern, '')
            category_id = get_catId(id)
            category_title = get_catTitle(category_id)

            df_record.loc[idx_rec] = [title, channel, running_time, 
                                      watch_date, watch_time, category_id, category_title, watched, account_2]  
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Watch Record
1: Ad
2: Watch Record
3: Ad
4: Watch Record
5: Ad
6: Watch Record
7: Watch Record
8: Ad
9: Watch Record
10: Ad
11: Watch Record
12: Watch Record
13: Watch Record
14: Ad
15: Search Record
16: Watch Record
17: Ad
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Ad
23: Watch Record
24: Ad
25: Watch Record
26: Watch Record
27: Watch Record
28: Watch Record
29: Watch Record
30: Watch Record
31: Ad
32: Ad
33: Watch Record
34: Ad
35: Search Record
36: Watch Record
37: Search Record
38: Ad
39: Watch Record
40: Ad
41: Community Post 	 Watch time: 오전 8:07
42: Watch Record
43: Community Post 	 Watch time: 오전 7:51
44: Search Record
45: Search Record
46: Search Record
47: Search Record
48: Search Record
49: Search Record
50: Search Record
51: Community Post 	 Watch time: 오전 7:51
52: Search Record
53: Search Record
54: Watch Record
55: Ad
56: Watch Record
57: Ad
58: Search Record
59: Search Record
60: Watch Record
61: Watch Record
62: Ad
63: Watch Record
64: Wa

AttributeError: 'NoneType' object has no attribute 'text'

In [36]:
content.find('a')

<a aria-label="‘삭제된 동영상을 시청함’ 활동 세부정보 열기" class="WFTFcf" role="button" tabindex="0">세부정보</a>

In [30]:
df_record.to_csv("data/watch_record_initial.csv", encoding='utf-8')
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account
0,"[여의도] 연봉 얼마 받으세요? | 직무,연차별 현실 연봉④",캐치TV,3:39,2023-02-19,오전 9:49,27,Education,1.00,woghks.study
1,몇 년째 인생이 달라진 게 없다면,말많은소녀,10:57,2023-02-19,오전 9:44,27,Education,1.00,woghks.study
2,"[한눈에 이슈] ""그동안 감사했습니다...지금까지 대한민국이었습니다"" 이 말이 곧 ...",KBS News,17:39,2023-02-19,오전 9:39,25,News & Politics,0.77,woghks.study
3,(30대 중반이 말하는) 돈과 진로에 관해 20대가 착각하는 것들,드로우앤드류,15:02,2023-02-19,오전 9:30,27,Education,1.00,woghks.study
4,이건 정말 아닌 것 같습니다.,북토크,11:46,2023-02-19,오전 9:28,22,People & Blogs,0.36,woghks.study


In [31]:
df_search.to_csv('data/search_record_initial.csv', encoding='utf-8')
df_search


,searchword,search_date,search_time
0,4평 역세권 주택,2023-02-16,오후 1:11
1,lck 매치 41,2023-02-16,오전 8:27
2,오너 니달리 펜타,2023-02-16,오전 8:24
3,용팔이,2023-02-13,오후 3:32
4,용훈,2023-02-13,오후 3:32
...,...,...,...
432,소맥거핀,2022-10-18,오후 1:34
433,stack que crashcourse,2022-10-18,오전 10:34
434,stack que,2022-10-18,오전 10:27
435,witcher 3 secondary quests,2022-10-17,오전 12:37


In [32]:
print(
    f"입력한 시청기록 레코드는 {num_target_data}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")


입력한 시청기록 레코드는 3000개, 
실제 시청한 영상은 1183개, 
광고는 1164개입니다.


In [37]:
driver.close()

### Notes (라고 쓰고 Code recycle bin으로 읽자)

In [ ]:
# 3-3) 입력한 날짜 이전의 레코드

# # 입력한 날짜까지의 레코드 호출
# target_date = get_date_data()

# btn_calendar = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/div[2]/div[2]/span/div[2]/c-wiz[1]/div/div/div/div/div[1]/button')
# btn_calendar.click()

# input_date = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'input')))
# input_date.send_keys(target_date)

# btn_apply = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/div[9]/div/div[2]/span/div[2]/div/c-wiz/div/div[4]/div/div/div[2]/div/button/div[3]')))
# btn_apply.click()